# 사전 준비

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.7 MB/s 
     |████████████████████████████████| 182 kB 76.3 MB/s 
     |████████████████████████████████| 7.6 MB 42.7 MB/s 


**경고 메시지 끄기**

In [2]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

**KcELECTRA, KoGPT2 모델과 KcELECTRA 토크나이저 불러오기**

In [3]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, GPT2LMHeadModel

tokenizer = ElectraTokenizer.from_pretrained("beomi/KcELECTRA-base")   # 토크나이저는 KcELECTRA에 사용한 토크나이저로 고정

ELECTRA_model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base")   # 비속어 감지의 사전학습 모델로는 KcELECTRA 사용
GPT2_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')   # 챗봇의 사전학습 모델로는 KoGPT2 사용

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

# 데이터 불러오기

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import urllib.request

In [5]:
curse_data = pd.read_table("/content/drive/Othercomputers/내 컴퓨터/Curse_Detection_Chatbot/Curse-detection-data/dataset.txt", 
                           names=["text", "label"], sep="|", header=None)
curse_data

,text,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5819,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5820,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5821,너가 한 말 중에,0
5822,제갈대중 ㅇㅂ,0


In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
chat_data = pd.read_csv('ChatBotData.csv')

chat_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


# 텍스트 전처리

**전처리 함수 정의**

In [7]:
!pip install soynlp
!pip install emoji==1.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 15.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 12.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=6efb291c1bc85731d2758f2698e4f0ee358d703ee7828882fd0f1a35de6bbec1
  Stored in directory: /root/.cache/pip/wheels/5e/8c/80/c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


In [8]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [9]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [10]:
def clean(x):
    x = pattern.sub(' ', x)         # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)      # URL 제거
    x = x.strip()                   # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

**전처리 및 데이터 분할**

텍스트 전처리는 비속어 감지 데이터에만 적용한다. (챗봇 데이터와 달리 댓글의 성격을 띄기 떄문에 무의미한 반복의 텍스트가 존재하기 때문)

In [11]:
import pandas as pd

In [12]:
# 비속어 감지 데이터
# train : validation = 3 : 1 (5824 = 2^6 * 7 * 13)

train_curse_text = [clean(curse_data['text'][idx]) for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_text = [clean(curse_data['text'][idx]) for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

train_curse_label = [curse_data['label'][idx] for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_label = [curse_data['label'][idx] for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

In [13]:
# 챗봇 데이터
# train : validation = 5 : 2 (11823 = 3 * 7 * 563)

train_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (0, int((chat_data.shape[0]/7)*5)))
val_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (int((chat_data.shape[0]/7)*5), int((chat_data.shape[0]/7)*7)))

# 데이터 구축, 토크나이징

**스페셜 토큰 추가**

In [14]:
special_tokens_dict = {'additional_special_tokens': ['[USR]', '[SYS]']}

tokenizer.add_special_tokens(special_tokens_dict)      # 딕셔너리형태의 스페셜 토큰을 추가

2

In [15]:
tokenizer.get_added_vocab()     # 새롭게 추가된 토큰의 인덱스 확인

{'[USR]': 50135, '[SYS]': 50136}

In [16]:
tokenizer.all_special_tokens    # 스페셜 토큰 확인

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[USR]', '[SYS]']

In [17]:
GPT2_model.resize_token_embeddings(len(tokenizer))   # 챗봇데이터에 쓰는 토큰이니 KoGPT2모델의 embedding layer를 업데이트 한다.

Embedding(50137, 768)

**비속어 감지 데이터**

In [18]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [19]:
# 토크나이징

train_input_token = tokenizer(train_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")
val_input_token = tokenizer(val_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")

In [20]:
# 비속어 감지 데이터셋

class CurseDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [21]:
train_curse_dataset = CurseDataset(train_input_token, train_curse_label)
val_curse_dataset = CurseDataset(val_input_token, val_curse_label)

In [22]:
# 데이터로더

train_curse_loader = DataLoader(train_curse_dataset, shuffle=True, batch_size=8)
val_curse_loader = DataLoader(val_curse_dataset, shuffle=True, batch_size=8)

**챗봇 데이터**

In [23]:
# 챗봇 데이터셋 (토크나이징 포함)

class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=64):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "[USR]"           # 새로 추가된 special token을 사용
        self.a_token = "[SYS]"
        self.cls = tokenizer.cls_token
        self.sep = tokenizer.sep_token
        self.mask = tokenizer.mask_token
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        index = self._data.iloc[idx]

        q = index["Q"]  # 질문
        q_toked = self.tokenizer.tokenize(self.cls + self.q_token + q)      #   질문
        q_len = len(q_toked)

        a = index["A"]  # 답변
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.sep)      #  답 
        a_len = len(a_toked)

        # 질문의 길이가 최대길이보다 클때
        if q_len > self.max_len: 
            q_toked = q_toked[-(int(self.max_len / 2)):]   # 질문길이를 최대길이의 반으로 
            q_len = len(q_toked)

        # 질문 + 답변 길이가 최대길이보다 클때
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        # 답변의 길이 = 최대길이 - 질문길이

            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              # 답변의 길이를 최대길이 - 질문길이
                
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 질문 + 답변 토큰을 index로 변환   
        token = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 padding
        while len(token) < self.max_len:
            token += [self.tokenizer.pad_token_id]

        # attention(어텐션마스크) = 질문+답변 길이 1 + 나머지(패딩) 0
        attention = [1]*(q_len+a_len) + [0]*(self.max_len - q_len - a_len)

        # token_type_ids(세그먼트 정보) = 질문 0 + 답변 1 + 나머지 0
        token_type = [0]*q_len + [1]*a_len + [0]*(self.max_len - q_len - a_len)

        label = q_toked[0:2] + [self.mask,]*(q_len-2) + a_toked[0:]
        # label을 index로 변환
        label = self.tokenizer.convert_tokens_to_ids(label)
        # 최대길이만큼 padding
        while len(label) < self.max_len:
            label += [self.tokenizer.pad_token_id]

        
        # 질문 + 답변, 어텐션마스크, 세그먼트 정보, 답변
        return (token, attention, token_type, label)

In [24]:
train_chat_dataset = ChatbotDataset(train_chat_data, max_len=64)
val_chat_dataset = ChatbotDataset(val_chat_data, max_len=64)

In [25]:
# collate_fn 구성

def collate_batch(batch):
    token_ids = [item[:][0] for item in batch]
    attention_mask = [item[:][1] for item in batch]
    token_tpye_ids = [item[:][2] for item in batch]
    label_ids = [item[:][3] for item in batch]

    return torch.LongTensor(token_ids), torch.LongTensor(attention_mask), torch.LongTensor(token_tpye_ids), torch.LongTensor(label_ids)

In [26]:
# 데이터로더

train_chat_loader = DataLoader(train_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)
val_chat_loader = DataLoader(val_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)

# 모델 학습

**모델 파라미터 설정**

In [27]:
import torch.nn.functional as F
from tqdm.auto import tqdm as tqdm_auto
from sklearn.metrics import accuracy_score

In [28]:
# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
# KcELECTRA 파라미터 설정
ELECTRA_epochs = 5
ELECTRA_learning_rate = 5e-5

ELECTRA_optimizer = torch.optim.AdamW(ELECTRA_model.parameters(), lr=ELECTRA_learning_rate)
ELECTRA_criterion = torch.nn.CrossEntropyLoss()

ELECTRA_step = 0
ELECTRA_eval_steps = len(train_curse_loader)        # 훈련 배치수

In [30]:
# KoGPT2 파라미터 설정
GPT2_epoch = 15
GPT2_learning_rate = 3e-5

GPT2_optimizer = torch.optim.AdamW(GPT2_model.parameters(), lr=GPT2_learning_rate)
GPT2_scheduler = torch.optim.lr_scheduler.ExponentialLR(GPT2_optimizer, gamma=0.9)

GPT2_step = 0
GPT2_eval_steps = len(train_chat_loader)        # 훈련 배치수

In [31]:
ELECTRA_model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [32]:
GPT2_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50137, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

**KcELECTRA 학습 진행**

In [33]:
for epoch in range(ELECTRA_epochs):
    n = 0
    train_accuracy = 0
    loss = 0
    train_loss = 0.0
    
    ELECTRA_model.train()
    for batch in tqdm_auto(train_curse_loader, mininterval=0.01, leave=True):
        ELECTRA_optimizer.zero_grad()     # 그래디언트 초기화

        # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = ELECTRA_model(**inputs).logits    # 모형으로 결과 예측

        loss = ELECTRA_criterion(outputs, labels)
        train_loss += loss
        
        loss.backward()
        ELECTRA_optimizer.step()

        ELECTRA_step += 1
        if ELECTRA_step % ELECTRA_eval_steps == 0:  # eval_steps 마다 loss를 출력
            i = 0
            val_accuracy = 0

            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                ELECTRA_model.eval()        # 평가모드로 전환

                for val_batch in tqdm_auto(val_curse_loader, mininterval=0.01, leave=True):

                    # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
                    val_labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
                    val_outputs = ELECTRA_model(**inputs).logits     # 모형으로 결과 예측

                    loss = ELECTRA_criterion(val_outputs, val_labels)  
                    val_loss += loss

                    val_accuracy += accuracy_score(val_labels.cpu(), val_outputs.argmax(dim=1).cpu())
                    i += 1

                avg_val_loss = val_loss / len(val_curse_loader)

            val_accuracy /= i
            print('Step %d, validation loss: %.4f, accuracy_score: %.2f' % (ELECTRA_step, avg_val_loss, val_accuracy))
            
        avg_train_loss = train_loss / len(train_curse_loader)
        train_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
        n += 1

    train_accuracy /= n
    print('epoch %d, train loss: %.4f, accuracy_score: %.2f \n' % (epoch, avg_train_loss, train_accuracy))

  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

Step 546, validation loss: 0.0570, accuracy_score: 1.00
epoch 0, train loss: 0.3771, accuracy_score: 0.84 



  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

...

  0%|          | 0/182 [00:00<?, ?it/s]

Step 2184, validation loss: 0.1664, accuracy_score: 0.88
epoch 3, train loss: 0.0852, accuracy_score: 0.98 



  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

Step 2730, validation loss: 0.0027, accuracy_score: 1.00
epoch 4, train loss: 0.0769, accuracy_score: 0.98 



**KoGPT2 학습 진행**

In [34]:
# GPU 캐시 비우기 (GPU 메모리 확보)
torch.cuda.empty_cache()

In [37]:
for epoch in range(GPT2_epoch):
    n = 0
    train_accuracy = 0
    loss = 0
    train_loss = 0.0
    
    for batch_idx, samples in enumerate(tqdm_auto(train_chat_loader, mininterval=0.01, leave=True)):
        GPT2_optimizer.zero_grad()       # optimizer 초기화(Gradient)

        # 모델 입력 텐서 GPU에 올리기
        token_ids, attention_mask, token_type_ids, label_ids = samples
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        label_ids = label_ids.to(device)

        out = GPT2_model(
            input_ids=token_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=label_ids,
            )
        
        loss = out.loss
        loss.backward()
        GPT2_optimizer.step()

        train_loss += loss.item()

    train_loss /= batch_idx
    GPT2_scheduler.step()
    
    print(f'epoch: {epoch}, train loss: {train_loss} \n')

  0%|          | 0/528 [00:00<?, ?it/s]

epoch: 0, train loss: 0.8311045692360605 



  0%|          | 0/528 [00:00<?, ?it/s]

epoch: 1, train loss: 0.5362522755334228 



  0%|          | 0/528 [00:00<?, ?it/s]

...

epoch: 12, train loss: 0.004368387822629937 



  0%|          | 0/528 [00:00<?, ?it/s]

epoch: 13, train loss: 0.004073527449630625 



  0%|          | 0/528 [00:00<?, ?it/s]

epoch: 14, train loss: 0.003516398017927368 

